## Testing heatmap labels generators

In [ ]:
import os
import sys
sys.path.insert(0, os.path.join('..','src')) 
import network
import utils
import random
from matplotlib import pyplot as plt
import scipy
from scipy.misc import imresize
import cv2
import numpy as np
%matplotlib inline

data_type = 'heatmap_crops'
#data_type = 'region_crops'
prediction_class_type = 'odm'
#prediction_class_type = 'single'
validate = True
class_balancing = False
input_shape = (224, 224, 3)
mini_batch_size = 128
crop_size = 400
arch = 'xception'
input_weights_name = 'xception-lay86-heatmap_crops-ep000-tloss0.0068-vloss0.0069.hdf5'
tl = network.TransferLearningSeaLionHeatmap(data_type = data_type, input_shape = input_shape, prediction_class_type = prediction_class_type, class_balancing= class_balancing, mini_batch_size=mini_batch_size, validate = validate)
#tl = network.TransferLearningSeaLionOrNoSeaLion(data_type = data_type, input_shape = input_shape, prediction_class_type = prediction_class_type, class_balancing= class_balancing, mini_batch_size=mini_batch_size, validate = validate)
tl.build(arch, input_shape = input_shape)
tl.load_weights(input_weights_name)

In [ ]:
x_batch, y_batch = tl.val_iterator.__next__()
print(y_batch.mean(), x_batch.mean())

for i in range(mini_batch_size):   
    x = x_batch[i]#
    pred = tl.model.predict(np.expand_dims(x, axis = 0))[0].reshape((80, 80))
    plt.subplot(1,3,1)
    plt.imshow(x)
    scipy.misc.imsave('delete/'+str(i)+'_x.jpg',x)
    plt.subplot(1,3,2)
    plt.imshow(pred, cmap = 'gray')
    scipy.misc.imsave('delete/'+str(i)+'_pred.jpg', scipy.misc.imresize(pred.reshape((80, 80)), (224,224)))
    plt.title(str(pred.mean()))
    plt.subplot(1,3,3)
    plt.imshow(y_batch[i].reshape((80, 80)), cmap = 'gray')
    scipy.misc.imsave('delete/'+str(i)+'_ground.jpg', scipy.misc.imresize(y_batch[i].reshape((80, 80)), (224,224)))
    plt.title(str(y_batch[i].reshape((80, 80)).mean()))
    #plt.show()
    

# Test images heatmap predictor

In [1]:
import os
import time
import sys
sys.path.insert(0, os.path.join('..','src')) 
import network
import utils
import random
from matplotlib import pyplot as plt
import scipy
import settings
import data
from scipy.misc import imresize
import cv2
import numpy as np
%matplotlib inline

data_type = 'original_test'
#data_type = 'region_crops'
prediction_class_type = 'odm'
#prediction_class_type = 'single'
validate = False
class_balancing = False
input_shape = (224, 224, 3)
batch_size = 1
crop_size = 400
arch = 'xception'
input_weights_name = 'xception-lay86-heatmap_crops-ep000-tloss0.0068-vloss0.0069.hdf5'
tl = network.TransferLearningSeaLionHeatmap(data_type = data_type, input_shape = input_shape, prediction_class_type = prediction_class_type, class_balancing= class_balancing, mini_batch_size=batch_size, validate = validate)
#tl = network.TransferLearningSeaLionOrNoSeaLion(data_type = data_type, input_shape = input_shape, prediction_class_type = prediction_class_type, class_balancing= class_balancing, mini_batch_size=mini_batch_size, validate = validate)
tl.build(arch, input_shape = input_shape)
tl.load_weights(input_weights_name)

loader = data.Loader()
test_data = loader.load_original_images(dataset = 'test_st1')
iterator = data.DataIterator(test_data, data_transformation = data.LoadTransformer(), batch_size = batch_size, shuffle = False, seed = None, class_balancing = False, class_transformation = lambda x: x)

Using TensorFlow backend.
2017-06-13 16:49:09,707 - noaa - INFO - Starting...
2017-06-13 16:49:09,708 - noaa.data - DEBUG - Loading train image counts
2017-06-13 16:49:09,720 - noaa.data - DEBUG - Loading train image coordinates
2017-06-13 16:49:10,317 - noaa.data - DEBUG - Loading train image mismatch labels
2017-06-13 16:49:10,321 - noaa - INFO - Resizing images deactivated
2017-06-13 16:49:10,328 - noaa.data - DEBUG - Loading stage 1 test set original images
2017-06-13 16:49:15,165 - noaa - INFO - Building network using xception as the pretrained architecture...
../src/network.py:338: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=Tensor("in...)`
  self.model = keras.models.Model(input=self.base_model.input, output=predictions)
2017-06-13 16:49:15,263 - noaa - INFO - Output shape of first layer is (None, 224, 224, 3)
2017-06-13 16:49:15,264 - noaa - INFO - Output shape of last layer is (None, 80, 80, 1)
2017-06-13 16:49:16,393 - noaa -

In [ ]:
for n in range(100):
    t0 = time.time()
    test_image = iterator.__next__()[0]
    test_image = test_image / test_image.max()
    #print(test_image.mean())
    plot = False
    if plot:
        plt.figure()
        plt.imshow(test_image)
        plt.show()
    nrows = int(test_image.shape[0]/crop_size)
    ncolumns = int(test_image.shape[1]/crop_size)
    full_obm = []
    settings.logger.info("Going for image with shape "+str(test_image.shape))
    total = float((nrows * ncolumns))
    count = 0.0
    for row in range(nrows):
        row_obms = []
        for column in range(ncolumns):
            if count%8==0:
                settings.logger.info(str(100*count/total)[:4]+"% completed")

            crop = utils.crop_image(test_image, (column*crop_size, row*crop_size), crop_size)
            crop = scipy.misc.imresize(crop, input_shape)
            if crop.max() > 0:
                crop = crop / crop.max()

            crop = np.expand_dims(crop, axis = 0)

            obm = tl.model.predict(crop)
            obm = np.squeeze(obm)
            row_obms.append(obm)
            count += 1
        row_obms = np.hstack(row_obms)
        full_obm.append(row_obms)  

    full_obm = np.vstack(full_obm)
    #print(full_obm.shape, full_obm.max(),full_obm.mean(),full_obm.min())
    if plot and 0:
        plt.figure()
        plt.imshow(np.squeeze(full_obm), cmap = 'gray')
        plt.show()
    trunc_img = test_image[:crop_size*nrows,:crop_size*ncolumns]
    trunc_img = scipy.misc.imresize(trunc_img, (full_obm.shape[0],full_obm.shape[1],3))
    trunc_img = trunc_img / trunc_img.max()
    red_obm = np.zeros((full_obm.shape[0],full_obm.shape[1],3))
    red_obm[:,:,0] = full_obm
    red_obm = red_obm / red_obm.max()
    img_sum = cv2.addWeighted(src1 = trunc_img, alpha = 1, src2 = red_obm, beta = 0.6, gamma = 0.001)
    img_sum = img_sum / img_sum.max()
    if plot:
        plt.figure()
        plt.imshow(img_sum)
        plt.show()
    scipy.misc.imsave('delete/'+str(n)+'.jpg',img_sum)
    settings.logger.info("... completed in "+str(time.time()-t0)+" seconds")

2017-06-13 16:52:42,048 - noaa - INFO - Going for image with shape (3840, 5760, 3)
2017-06-13 16:52:42,051 - noaa - INFO - 0.0% completed
2017-06-13 16:53:06,128 - noaa - INFO - 6.34% completed
2017-06-13 16:53:36,982 - noaa - INFO - 12.6% completed
2017-06-13 16:54:03,006 - noaa - INFO - 19.0% completed
2017-06-13 16:54:23,357 - noaa - INFO - 25.3% completed
2017-06-13 16:54:37,742 - noaa - INFO - 31.7% completed
2017-06-13 16:54:51,987 - noaa - INFO - 38.0% completed
2017-06-13 16:55:06,882 - noaa - INFO - 44.4% completed
2017-06-13 16:55:16,974 - noaa - INFO - 50.7% completed
2017-06-13 16:55:30,517 - noaa - INFO - 57.1% completed
2017-06-13 16:55:42,335 - noaa - INFO - 63.4% completed
2017-06-13 16:55:53,428 - noaa - INFO - 69.8% completed
2017-06-13 16:56:07,056 - noaa - INFO - 76.1% completed
2017-06-13 16:56:20,419 - noaa - INFO - 82.5% completed
2017-06-13 16:56:35,407 - noaa - INFO - 88.8% completed
2017-06-13 16:56:48,791 - noaa - INFO - 95.2% completed
2017-06-13 16:57:23,69

# ...

In [ ]:
import os
import sys
sys.path.insert(0, os.path.join('..','src')) 
import network
import utils
import random
from matplotlib import pyplot as plt
import scipy
from scipy.misc import imresize
%matplotlib inline

data_type = 'original_test'
validate = False
input_shape = (224, 224, 3)
mini_batch_size = 1
crop_size = 400
arch = 'resnet'
input_weights_name = 'resnet-lay107-ep002-tloss0.0134-vloss0.0166.hdf5'
tl = network.TransferLearningSeaLionOrNoSeaLion(data_type = data_type, input_shape = input_shape, prediction_class_type = "single", mini_batch_size=mini_batch_size, validate = validate)
tl.build(arch, input_shape = input_shape, summary = False)
tl.load_weights(input_weights_name)



## load one test image

In [ ]:
test_image = tl.iterator.__next__()[0]
plt.figure()
plt.imshow(test_image)
plt.title(str(test_image.shape))
plt.show()


## positives

In [ ]:
coordinates = [3330, 1100]
step_px = 50
samples = 20
for i in range(samples):
    crop = utils.crop_image(test_image, tuple(coordinates), crop_size)
    crop = scipy.misc.imresize(crop,(224,224,3))
    pred = tl.model.predict(crop.reshape((1,224,224,3)))[0][0]
    plt.figure()
    plt.imshow(crop)
    plt.title(str(pred))
    coordinates[1] = coordinates[1] + step_px
    

## negatives

In [ ]:
for i in range(samples):
    coordinates = (random.randint(0,test_image.shape[1]-crop_size), random.randint(0,test_image.shape[0]-crop_size))
    crop = utils.crop_image(test_image, coordinates, crop_size)
    crop = scipy.misc.imresize(crop,(224,224,3))
    plt.figure()
    plt.imshow(crop)
    pred = tl.model.predict(crop.reshape((1,224,224,3)))[0][0]
    plt.title(str(pred))
    plt.show()
